In [836]:
from starter import *
import pickle
import datetime
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Load data

In [2]:
train = pd.read_csv('train.csv')
holidays = pd.read_csv('holidays.csv')
weather = pd.read_csv('weather.csv')
metadata = pd.read_csv('metadata.csv')
submission_format = pd.read_csv('submission_format.csv')
submission_frequency = pd.read_csv('submission_frequency.csv')

In [546]:
submission_frequency = pickle.load(open('submission_frequency.p', 'rb'))
metadata = pickle.load(open('metadata.p', 'rb'))
holidays = pickle.load(open('holidays.p', 'rb'))
weather_grouped = pickle.load(open('weather_grouped.p', 'rb'))
train = pickle.load(open('train.p', 'rb'))
submission_format = pickle.load(open('submission_format.p', 'rb'))
df = pickle.load(open('df.p', 'rb'))

## Feature Engineering

In [244]:
df['Holiday'].fillna(0, inplace=True)
df['isHoliday'].fillna(0, inplace=True)
df.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1, inplace=True)

In [48]:
df.head()

,obs_id,SiteId,Timestamp,Value,Date,Time,Hour,DayofWeek,Year,Month,Temperature,Distance,Holiday,isHoliday
0,744519,1,2014-09-03,9.096555e+05,2014-09-03,00:00:00,0,Wednesday,2014,9,19.60,22.921092,0,0.0
1,7627564,1,2014-09-04,1.748273e+06,2014-09-04,00:00:00,0,Thursday,2014,9,21.30,22.921092,0,0.0
2,7034705,1,2014-09-05,0.000000e+00,2014-09-05,00:00:00,0,Friday,2014,9,23.35,22.921092,0,0.0
3,5995486,1,2014-09-06,0.000000e+00,2014-09-06,00:00:00,0,Saturday,2014,9,21.60,22.921092,0,0.0
4,7326510,1,2014-09-07,0.000000e+00,2014-09-07,00:00:00,0,Sunday,2014,9,15.80,22.921092,0,0.0


In [245]:
submission_frequency.head()

,ForecastId,ForecastPeriodNS,ForecastPeriodMin,Period_Quarter,Period_Hour,Period_Days
0,1,86400000000000,1440.0,0,0,1
1,2,86400000000000,1440.0,0,0,1
2,3,86400000000000,1440.0,0,0,1
3,4,86400000000000,1440.0,0,0,1
4,5,3600000000000,60.0,0,1,0


In [267]:
# align SiteId to ForecastId
id_align = submission_format.groupby(by = 'ForecastId').mean()
id_align['ForecastId'] = id_align.index
id_align.drop(['obs_id', 'Value'], axis=1, inplace=True)

In [269]:
# merge SiteId to ForecastId
submission_frequency = submission_frequency.merge(id_align, how='left', on='ForecastId')
site_type = submission_frequency.groupby(by='SiteId').mean()

In [497]:
metadata.head()

,SiteId,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff,Temperature
SiteId,,,,,,,,,,,,
1,1,1387.205119,15.0,18.0,0,0,0,0,0,1,1,0
2,2,6098.278376,30.0,18.0,0,0,0,0,0,1,1,0
3,3,10556.293605,5.0,18.0,0,0,0,0,0,1,0,0
5,5,12541.181277,30.0,18.0,0,0,0,0,0,1,1,0
6,6,9150.195373,30.0,18.0,0,0,0,0,0,1,1,0


## Model for scoring

In [49]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [226]:
def RMSE(y, pred):
    return mean_squared_error(y, pred)**0.5

In [88]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [502]:
# acquire siteid data
def get_site_data(data=None, site_id=None, features=['Value','Temperature']):
    df = data[data.SiteId==site_id]
    df.index = df.Timestamp
    df = df[features]
    return df

In [505]:
# segment and interpolate data
def prepare_site(data=None, site_id=None, features= ['Value', 'Temperature'], method='linear'):
    df = get_site_data(data, site_id, features)
    try:
        # start where there isn't consecutive NaNs
        start_index = df[(df.Temperature.isnull()==False)&(df.Temperature.shift(-1).isnull()==False)].index[0]
        df = df.loc[start_index:,:]
        df['Temperature'].interpolate(method, inplace=True)
    except:
        print('No temperature data')
    return df

In [510]:
df2=prepare_site(df, 2)

In [458]:
def create_master_data(data=None, sites=None, features=['Value', 'Temperature'], method='linear'):
    master_df = pd.DataFrame()
    for site in sites:
        threshold_size = data[data.SiteId==site].shape[0]*0.5
        site_features = features.copy()
        if data[data.SiteId==site].Temperature.isnull().value_counts()[0] < threshold_size:
            site_features.remove('Temperature')
        df = prepare_site(data, site, site_features)
        print('Finished site {} with rows {}'.format(site, df.shape[0]))
        master_df = pd.concat([master_df, df], axis=0)
    print('Final rows: {}'.format(master_df.shape[0]))
    return master_df

In [683]:
# normalize and reframe features
def reframe_features(data=None, n_in=1):
    values = data.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    # frame as supervised learning
    reframed = series_to_supervised(scaled, n_in, 1)
    reframed.drop(reframed.columns[[reframed.shape[1] - i for i in range(1,data.shape[1])]], axis=1, inplace=True)
    print(reframed.head())
    return reframed, scaler

In [512]:
reframed_data = reframe_features(df2, 2)

   var1(t-2)  var2(t-2)  var1(t-1)  var2(t-1)   var1(t)
2   0.201850   0.763602   0.222406   0.769231  0.215942
3   0.222406   0.769231   0.215942   0.769231  0.180349
4   0.215942   0.769231   0.180349   0.795497  0.167053
5   0.180349   0.795497   0.167053   0.769231  0.179651
6   0.167053   0.769231   0.179651   0.769231  0.157865


In [603]:
# split train/test sets from reframed data
def split_train_test_val(reframed):
    values = reframed.values
    n_train_days = int(len(values) * 0.7)
    train = values[:n_train_days, :]
    test = values[n_train_days:, :]
    # Split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # Reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    return train_X, train_y, test_X, test_y

In [604]:
train_X, train_y, test_X, test_y = split_train_test_val(reframed_data)

(24283, 1, 4) (24283,) (10407, 1, 4) (10407,)


In [713]:
# build multivariate LSTM model with validation
def create_LSTM_val(train_X=None, train_y=None, test_X=None, test_y=None, epochs=50, batch_size=100):
    model = Sequential()
    model.add(LSTM(4, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(train_X, train_y, epochs=epochs,
                                batch_size=batch_size, validation_data=(test_X, test_y),
                                verbose=1, shuffle=False)
    return model

In [529]:
multi_model = create_LSTM_val(train_X, train_y, test_X, test_y, epochs=50)

Train on 24283 samples, validate on 10407 samples
Epoch 1/50
24283/24283 [==============================] - 3s 108us/step - loss: 0.0663 - val_loss: 0.0045
Epoch 2/50
24283/24283 [==============================] - 1s 25us/step - loss: 0.0201 - val_loss: 0.0032
Epoch 3/50
24283/24283 [==============================] - 1s 27us/step - loss: 0.0122 - val_loss: 0.0025
Epoch 4/50
24283/24283 [==============================] - 1s 24us/step - loss: 0.0086 - val_loss: 0.0026
Epoch 5/50
24283/24283 [==============================] - 1s 24us/step - loss: 0.0078 - val_loss: 0.0027
Epoch 6/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0074 - val_loss: 0.0026
Epoch 7/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0071 - val_loss: 0.0026
Epoch 8/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0068 - val_loss: 0.0026
Epoch 9/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0065 - val_loss: 0.0025
Epoc

In [517]:
yhat = multi_model.predict(test_X)

In [518]:
# compare forecast to actual
def forecast_score(pred, actual_input, actual_output):
    actual_input = actual_input.reshape((actual_input.shape[0], actual_input.shape[2]))
    
    # Invert scaling for forecast
    inv_pred = np.concatenate((pred, actual_input[:, 1:]), axis=1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(inv_pred)
    inv_pred = scaler.inverse_transform(inv_pred)
    inv_pred = inv_pred[:,0]
    
    # Invert scaling for actual
    actual_output = actual_output.reshape((len(actual_output), 1))
    inv_output = np.concatenate((actual_output, actual_input[:, 1:]), axis=1)
    inv_output = scaler.inverse_transform(inv_output)
    inv_output = inv_output[:,0]
    rmsle = RMSE(inv_output, inv_pred)
    
    return rmsle

In [519]:
forecast_score(yhat, test_X, test_y)

0.016920891150341268

## Model for site prediction

In [862]:
submission_format[submission_format.ForecastId==4].head()

,obs_id,SiteId,Timestamp,ForecastId,Value,Date,Year,Month,Time,Hour,DayofWeek
180,1583559,1,2017-08-18,4,0.0,2017-08-18,2017,8,00:00:00,0,Friday
181,2071107,1,2017-08-19,4,0.0,2017-08-19,2017,8,00:00:00,0,Saturday
182,1253673,1,2017-08-20,4,0.0,2017-08-20,2017,8,00:00:00,0,Sunday
183,2325727,1,2017-08-21,4,0.0,2017-08-21,2017,8,00:00:00,0,Monday
184,4834576,1,2017-08-22,4,0.0,2017-08-22,2017,8,00:00:00,0,Tuesday


In [781]:
# concatenate train and test sets
def load_site(train=None, test=None, site_id=None, weather=weather_grouped, holidays=holidays):
    train = train[train.SiteId==site_id]
    test = test[test.SiteId==site_id]
    train.loc[:,'ForecastId'] = 0
    data = pd.concat([train, test], axis=0)
    data = data.merge(weather, how='left', on=['Timestamp', 'SiteId'])
    data = data.merge(holidays, how='left', on=['Date','SiteId'])
    data['Holiday'].fillna(0, inplace=True)
    data['isHoliday'].fillna(0, inplace=True)
    data.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1, inplace=True)
    return data

In [1188]:
# find start and end times for site
def find_start_end(data):
    time_pairs = []
    forecast_ids = data.ForecastId.unique()[1:]
    forecast_length = data[data.ForecastId==forecast_ids[0]].shape[0]-1
    timedelta = data.Timestamp[1] - data.Timestamp[0]
    train_start = data.Timestamp[0]
    test_end = data[data.ForecastId==forecast_ids[0]].Timestamp[forecast_length]
    test_start = test_end - forecast_length*timedelta
    train_end = test_start - timedelta
    time_pairs.append((train_start, train_end, test_start, test_end))
    
    for forecast_id in forecast_ids[1:]:
        forecast_length = data[data.ForecastId==forecast_id].shape[0]-1
        train_start = test_end + timedelta
        test_end = data[data.ForecastId==forecast_id].Timestamp[forecast_length]
        test_start = test_end - forecast_length*timedelta
        train_end = test_start - timedelta
        time_pairs.append((train_start, train_end, test_start, test_end))
    
    return time_pairs

In [1107]:
df1 = load_site(train, submission_format, 1)
df1.sort_values('Timestamp', inplace=True)
df1.index = df1.Timestamp

In [1189]:
time_pairs = find_start_end(df1)

In [1109]:
df1_1 = df1[(df1.Timestamp >= time_pairs[0][0]) & (df1.Timestamp <= time_pairs[0][3])]

In [1172]:
df302 = load_site(train, submission_format, 302)
df302.sort_values('Timestamp', inplace=True)
df302.index = df302.Timestamp

In [1179]:
df302 = prepare_data(df302, features=['Value', 'Temperature','isHoliday','ForecastId','Timestamp'])
time_pairs302 = find_start_end(df302)

In [1178]:
# segment and interpolate data for site
def prepare_data(data=None, features= ['Value', 'Temperature'], method='linear'):
    data.index = data.Timestamp
    data = data[features]
    data['Temperature'].interpolate(method, inplace=True)
    try:
        # start where there isn't consecutive NaNs
        start_index = data[(data.Temperature.isnull()==False)&(data.Temperature.shift(-1).isnull()==False)].index[0]
        data = data.loc[start_index:,:]
    except:
        print('No temperature data')
    return data

Change the original function up top

In [1111]:
# normalize and reframe features
def reframe_features(data=None, n_in=1):
    values = data.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    # frame as supervised learning
    reframed = series_to_supervised(scaled, n_in, 1)
    reframed.drop(reframed.columns[[reframed.shape[1] - i for i in range(1,data.shape[1])]], axis=1, inplace=True)
    print(reframed.head())
    return reframed, scaler

In [983]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
#     if dropnan:
#         agg.dropna(inplace=True)
    return agg

In [1112]:
df1_1 = prepare_data(df1_1, features=['Value','Temperature','isHoliday', 'ForecastId'])

In [1113]:
df1_1.loc[:,'Timestamp'] = df1_1.index

In [1114]:
df1_1.columns

Index(['Value', 'Temperature', 'isHoliday', 'ForecastId', 'Timestamp'], dtype='object')

In [1115]:
df1_1.drop(['ForecastId','Timestamp'],axis=1, inplace=True)
n_train_days = df1_1[(df1_1.index >=time_pairs[0][0]) & (df1_1.index <=time_pairs[0][1])].shape[0]-1
n_features = df1_1.shape[1]
reframed_data, scaler = reframe_features(df1_1, 2)

   var1(t-2)  var2(t-2)  var3(t-2)  var1(t-1)  var2(t-1)  var3(t-1)   var1(t)
0        NaN        NaN        NaN        NaN        NaN        NaN  0.091209
1        NaN        NaN        NaN   0.091209   0.846487        0.0  0.175295
2   0.091209   0.846487        0.0   0.175295   0.883243        0.0  0.000000
3   0.175295   0.883243        0.0   0.000000   0.927568        0.0  0.000000
4   0.000000   0.927568        0.0   0.000000   0.889730        0.0  0.000000


In [1116]:
reframed_data.fillna(0, inplace=True)

In [1117]:
n_train_days, n_features

(359, 3)

In [1120]:
# split train/test sets from reframed data
def split_train_test(reframed, n_train_days):
    values = reframed.values
    train = values[:n_train_days, :]
    test = values[n_train_days+1:, :]
    # Split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    # Reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    return train_X, train_y, test_X, test_y

In [1121]:
# build multivariate LSTM model for predictions
def create_LSTM(train_X=None, train_y=None, epochs=50, batch_size=100):
    model = Sequential()
    model.add(LSTM(4, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(train_X, train_y, epochs=epochs,
                                batch_size=batch_size,
                                verbose=1, shuffle=False)
    return model

In [1118]:
train_X, train_y, test_X, test_y = split_train_test(reframed_data, n_train_days)

(359, 6) (359,) (60, 6) (60,)
(359, 1, 6) (359,) (60, 1, 6) (60,)


In [1119]:
df1.loc[time_pairs[0][0]:time_pairs[0][3],:].shape

(420, 15)

In [1122]:
df1_model = create_LSTM(train_X, train_y, epochs=50)

Epoch 1/50
359/359 [==============================] - 4s 11ms/step - loss: 0.2835
Epoch 2/50
359/359 [==============================] - 0s 41us/step - loss: 0.2733
Epoch 3/50
359/359 [==============================] - 0s 47us/step - loss: 0.2633
Epoch 4/50
359/359 [==============================] - 0s 49us/step - loss: 0.2536
Epoch 5/50
359/359 [==============================] - 0s 46us/step - loss: 0.2440
Epoch 6/50
359/359 [==============================] - 0s 39us/step - loss: 0.2347
Epoch 7/50
359/359 [==============================] - 0s 34us/step - loss: 0.2257
Epoch 8/50
359/359 [==============================] - 0s 43us/step - loss: 0.2169
Epoch 9/50
359/359 [==============================] - 0s 35us/step - loss: 0.2083
Epoch 10/50
359/359 [==============================] - 0s 35us/step - loss: 0.2000
Epoch 11/50
359/359 [==============================] - 0s 36us/step - loss: 0.1920
Epoch 12/50
359/359 [==============================] - 0s 38us/step - loss: 0.1842
Epoch 13/50
3

In [1123]:
# make predictions with LSTM model
def model_predict(model, test_X, n_features, scaler):
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    inv_yhat = np.concatenate([yhat, test_X], axis=1)[:,:n_features]
    inv_yhat = scaler.inverse_transform(inv_yhat)[:,0]
    return inv_yhat

In [1124]:
inv_yhat = model_predict(df1_model, test_X, n_features, scaler)

In [1125]:
inv_yhat.shape[0]

60

In [1126]:
# join predictions to dataframe
df1_1.loc[time_pairs[0][2]:time_pairs[0][3], 'Value'] = inv_yhat

In [1102]:
df1_1

,Value,Temperature,isHoliday
Timestamp,,,
2014-09-03,9.096555e+05,19.60,0.0
2014-09-04,1.748273e+06,21.30,0.0
2014-09-05,0.000000e+00,23.35,0.0
2014-09-06,0.000000e+00,21.60,0.0
2014-09-07,0.000000e+00,15.80,0.0
2014-09-08,1.964878e+06,15.25,0.0
2014-09-09,3.266904e+06,19.15,0.0
2014-09-10,2.926094e+06,19.25,0.0
2014-09-11,1.720502e+06,23.55,0.0


In [1210]:
#adds prediction to proper submission format
def predict_site(train=None, test=None, site_id=None):
    #load site data and train/test times
    data = load_site(train, test, site_id)
    data.sort_values('Timestamp', inplace=True)
    data.index = data.Timestamp
    data = prepare_data(data, features=['Value', 'Temperature','isHoliday','ForecastId','Timestamp'])
    time_pairs = find_start_end(data)
    
    #build model and predict for each forecastid
    for n, times in enumerate(time_pairs):
        df = data[(data.Timestamp >= time_pairs[n][0]) & (data.Timestamp <= time_pairs[n][3])]
        df = prepare_data(df, features=['Value','Temperature','isHoliday','ForecastId'])
        df['Temperature'].fillna(0, inplace=True)
        df.loc[:,'Timestamp'] = df.index
        df.drop(['ForecastId','Timestamp'],axis=1, inplace=True)
        n_train_days = df[(df.index >=time_pairs[n][0]) & (df.index <=time_pairs[n][1])].shape[0]-1
        n_features = df.shape[1]
        reframed_data, scaler = reframe_features(df, 2)
        reframed_data.fillna(0, inplace=True)
        train_X, train_y, test_X, test_y = split_train_test(reframed_data, n_train_days)
        model = create_LSTM(train_X, train_y, epochs=50)
        inv_yhat = model_predict(model, test_X, n_features, scaler)
        data.loc[time_pairs[n][2]:time_pairs[n][3], 'Value'] = inv_yhat
        
    return data

In [ ]:
# add features from metadata to site data
def add_meta(data, site_id):
    